In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
log_name = 'test'
model_name = 'pcr'
n_processes = 20

with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)


known_activities = ['Callback timeout', 'Export result', 'Export to EMS', 'Match patient data', 'Receive sample state', 'Send notification', 'Wait for plate validation', 'timeout']

likelihoods_A = None
likelihoods_A_S = None
likelihoods_A_S_AC = None

/tmp/ipykernel_32761/3766840309.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                 | 0/1235 [00:00<?, ?it/s]

  0%|                                       | 1/1235 [00:28<9:43:55, 28.39s/it]

  0%|                                       | 2/1235 [00:29<4:16:37, 12.49s/it]

  0%|                                       | 3/1235 [00:30<2:25:50,  7.10s/it]

  0%|▏                                      | 4/1235 [00:31<1:33:47,  4.57s/it]

...

 94%|██████████████████████████████████▊  | 1163/1235 [00:07<00:00, 166.13it/s]

 96%|███████████████████████████████████▍ | 1183/1235 [00:07<00:00, 166.79it/s]

 97%|████████████████████████████████████ | 1202/1235 [00:07<00:00, 170.14it/s]

 99%|████████████████████████████████████▌| 1221/1235 [00:07<00:00, 173.58it/s]

100%|█████████████████████████████████████| 1235/1235 [00:08<00:00, 153.32it/s]

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.6936919899819694448925467721')

In [6]:
np.mean(get_pscores(likelihoods_A))

np.float64(11836.900555009734)

In [7]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S = evaluator_A_S.sample_cases(False, True)

  0%|                                                 | 0/1235 [00:00<?, ?it/s]

  0%|                                      | 1/1235 [00:32<11:13:31, 32.75s/it]

  0%|                                       | 2/1235 [00:34<4:57:31, 14.48s/it]

  0%|                                       | 3/1235 [00:35<2:54:29,  8.50s/it]

  0%|▏                                      | 4/1235 [00:37<1:55:24,  5.62s/it]

...

 95%|███████████████████████████████████  | 1171/1235 [00:07<00:00, 150.36it/s]

 97%|███████████████████████████████████▋ | 1193/1235 [00:07<00:00, 160.13it/s]

 98%|████████████████████████████████████▎| 1214/1235 [00:07<00:00, 170.63it/s]

100%|████████████████████████████████████▉| 1232/1235 [00:07<00:00, 172.48it/s]

100%|█████████████████████████████████████| 1235/1235 [00:07<00:00, 154.55it/s]

In [8]:
np.mean([v.ln() for v in likelihoods_A_S[0].values()])

Decimal('0.6991436501097791789094899324')

In [9]:
np.mean(get_pscores(likelihoods_A_S))

np.float64(11041.441358390723)

In [10]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'day_of_week' : 'day_of_week',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False, True)

  0%|                                                 | 0/1235 [00:00<?, ?it/s]

  0%|                                      | 1/1235 [00:43<14:57:37, 43.64s/it]

  0%|                                       | 3/1235 [00:44<4:02:04, 11.79s/it]

  0%|▏                                      | 4/1235 [00:46<2:46:15,  8.10s/it]

  0%|▏                                      | 5/1235 [00:48<2:04:40,  6.08s/it]

...

 95%|███████████████████████████████████  | 1170/1235 [00:06<00:00, 194.62it/s]

 97%|███████████████████████████████████▋ | 1192/1235 [00:06<00:00, 197.72it/s]

 98%|████████████████████████████████████▎| 1212/1235 [00:06<00:00, 196.72it/s]

100%|████████████████████████████████████▉| 1232/1235 [00:07<00:00, 196.26it/s]

100%|█████████████████████████████████████| 1235/1235 [00:07<00:00, 173.57it/s]

In [11]:
np.mean([v.ln() for v in likelihoods_A_S_D[0].values()])

Decimal('1.199662881265988591257608905')

In [12]:
np.mean(get_pscores(likelihoods_A_S_D))

np.float64(8829.608237342816)

In [13]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_activity-count/',
                     strict_parser=False)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'known_activities' : known_activities,
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False, True)

  0%|                                                 | 0/1235 [00:00<?, ?it/s]

  0%|                                      | 1/1235 [00:36<12:21:02, 36.03s/it]

  0%|                                       | 2/1235 [00:43<6:30:54, 19.02s/it]

  0%|▏                                      | 5/1235 [00:44<1:54:55,  5.61s/it]

  1%|▎                                      | 8/1235 [00:47<1:07:41,  3.31s/it]

...

 94%|██████████████████████████████████▊  | 1161/1235 [00:07<00:00, 170.24it/s]

 96%|███████████████████████████████████▎ | 1180/1235 [00:07<00:00, 174.72it/s]

 97%|███████████████████████████████████▉ | 1198/1235 [00:07<00:00, 167.14it/s]

 98%|████████████████████████████████████▍| 1216/1235 [00:08<00:00, 164.65it/s]

100%|█████████████████████████████████████| 1235/1235 [00:08<00:00, 150.95it/s]

In [14]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [15]:
np.mean([v.ln() for v in likelihoods_A_S_AC[0].values()])

Decimal('0.3233638262255983455410566407')

In [16]:
np.mean(get_pscores(likelihoods_A_S_AC))

np.float64(15322.853189465139)

In [17]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)